In [2]:
import time
import datetime
import pandas as pd
import re

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException,TimeoutException,ElementNotInteractableException


### Task :

bina.az saytından Nərmanov rayonunda yerləşən Yeni tikililəri scrape etmək.

In [3]:
#------ browser parameters 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--blink-settings=imagesEnabled=false')#  disable image loading
chrome_options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2}) # or alternatively we can set direct preference:
#------ Functions
def repair_or_not(i): # Təmirlidir ya yox
    try:
        wait_repair = WebDriverWait(i, timeout=0.05,poll_frequency=0.001)
        wait_repair.until(EC.presence_of_element_located((By.CLASS_NAME,'repair')))
        repair='Təmirli'
    except TimeoutException:
        repair="Təmirli deyil"
    return repair

def bill(i): # Çıxarış var ya yox 
    try:
        wait_repair = WebDriverWait(i, timeout=0.05,poll_frequency=0.001)
        wait_repair.until(EC.presence_of_element_located((By.CLASS_NAME,'bill_of_sale')))
        bill_of_sale='Çıxarış var'
    except TimeoutException:
        bill_of_sale="Çıxarış yoxdur"
    return bill_of_sale

def mortgage(i): # İpoteka var ya yox
    try:
        wait_repair = WebDriverWait(i, timeout=0.05,poll_frequency=0.001)
        wait_repair.until(EC.presence_of_element_located((By.CLASS_NAME,'mortgage')))
        mort='İpoteka var'
    except TimeoutException:
        mort="İpoteka yoxdur"
    return mort

#------ requests
url = 'https://bina.az/baki/alqi-satqi/menziller/yeni-tikili'

browser=webdriver.Chrome(chrome_options=chrome_options)
browser.get(url)
print(browser.title)

#------ filter
browser.find_element(By.ID,'js-search-row-filters-btn').click()  # Click Ətraflı axtarış
time.sleep(0.5)
browser.find_element(By.XPATH,'//*[@id="js-search-filters-row-locations"]/div[2]/a[1]').click() # Rayon və qəsəbələr
time.sleep(0.5)
browser.find_element(By.XPATH,'//*[@id="js-search-locations"]/div/div[3]/div[2]/div[1]/div[1]/ul/li[6]/span[1]').click() # Nərimanov r. Rayonu dəyişmək ehtiyac olsa buradan XPATH-ı dəyişmək kifayətdir.
time.sleep(0.5)
browser.find_element(By.XPATH,'//*[@id="js-search-locations"]/div/div[3]/div[2]/div[2]/div[5]').click() # Axtarışa əlavə et
time.sleep(0.5)
elan_button =browser.find_element(By.XPATH,'//*[@id="js-search-filters"]/div[2]/button') # elan göstər
elan_button.click() 
filter_uzre_elan_sayi=int(re.search(r'\d+',elan_button.text)[0])
print(f'Filter üzrə elan sayı: {filter_uzre_elan_sayi}')
time.sleep(1)

#------- scraping
links, prices, currency, locations, informations, ad_times, scrape_time_ls, repair_, bill_, mortgage_ = [], [], [], [], [], [], [], [], [], []
page=1
page_error=0
while True:
    try:
        print("Page: ",page)
        # if loop==4:
        #     break
        try:
            items = browser.find_elements(By.CLASS_NAME,'items_list')[2].find_elements(By.CLASS_NAME,'items-i')
            scrape_time=datetime.datetime.now().strftime("%d/%m/%Y %H:%M") # scrape time
            iter=0
            for item in items:
                wait = WebDriverWait(item, timeout=4,poll_frequency=0.001)
                iter+=1
                print(iter,end=',')
                #--- find elements
                link     = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'item_link'))).get_attribute('href') # links.append( item.find_element(By.CLASS_NAME,'item_link').get_attribute('href')) # tag a: link
                price    = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'price-val'))).text # prices.append(item.find_element(By.CLASS_NAME,'price-val').text)
                curr     = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'price-cur'))).text # currency.append(item.find_element(By.CLASS_NAME,'price-cur').text)
                location = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'location'))).text # location.append(item.find_element(By.CLASS_NAME,'location').text)
                info     = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'name'))).text  # info.append(item.find_element(By.CLASS_NAME,'name').text)
                ad_time  = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'city_when'))).text
                repair   = repair_or_not(i=item)
                cixaris  = bill(i=item)
                mort     = mortgage(i=item)
                #----- append 
                links.append(link), prices.append(price), currency.append(curr), locations.append(location), informations.append(info),ad_times.append(ad_time),\
                repair_.append(repair),bill_.append(cixaris), mortgage_.append(mort)
                scrape_time_ls.append(scrape_time) 
            
            # break
                


        except StaleElementReferenceException:
            print('Stale Element Reference Exception occurred. Refreshing the page and trying again.')
            continue
        wait = WebDriverWait(browser, timeout=10,poll_frequency=0.01)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME,'next'))).click() # Növbəti səhifəyə keçid
        time.sleep(1.3)
        page+=1
        print()
    #---- end of the page
    except:
        print('\n','Texniki problem baş verdi !!!')
        page_error+=1
        if page_error==3:
            print('Səhifə bitti !!!')
            break
        continue
#------- end
time.sleep(1)
# browser.close()
#------- check data length
assert len(links) == len(prices) == len(currency) == len(locations) == len(informations) == len(ad_times) == len(scrape_time_ls)

/var/folders/hc/7yss87_95rg586gdpqc4xfsw0000gn/T/ipykernel_73938/2367956861.py:36: DeprecationWarning: use options instead of chrome_options
  browser=webdriver.Chrome(chrome_options=chrome_options)


Yeni tikili almaq, Bakı - bina.az
Filter üzrə elan sayı: 4164
Page:  1
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,
Page:  2
1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,
Page:  3
1,2,3,4,5,6,7,8,
 Texniki problem baş verdi !!!
Page:  3

 Texniki problem baş verdi !!!
Page:  3

 Texniki problem baş verdi !!!
Səhifə bitti !!!


# Data Frame

In [4]:
data=pd.DataFrame({'Links':links,
            'Qiymetler':prices,
            'Valyuta':currency,
            'Yer':locations,
            'Məlumat':informations,
            'Paylaşılma vaxtı':ad_times,
            'Scrape Vaxtı':scrape_time_ls,
            'Təmirli?':repair_,
            'Çıxarış?': bill_,
            'Ipoteka?':mortgage_})
data

,Links,Qiymetler,Valyuta,Yer,Məlumat,Paylaşılma vaxtı,Scrape Vaxtı,Təmirli?,Çıxarış?,Ipoteka?
0,https://bina.az/items/3425902,185 000,AZN,Gənclik m.,3 otaqlı\n65 m²\n1/4 mərtəbə,"Bakı, bugün 14:13",17/04/2023 14:14,Təmirli deyil,Çıxarış var,İpoteka yoxdur
1,https://bina.az/items/3290154,295 000,AZN,28 May m.,3 otaqlı\n130 m²\n12/14 mərtəbə,"Bakı, bugün 14:07",17/04/2023 14:14,Təmirli,Çıxarış var,İpoteka var
2,https://bina.az/items/3241518,309 000,AZN,Nəriman Nərimanov m.,3 otaqlı\n150 m²\n9/19 mərtəbə,"Bakı, bugün 14:07",17/04/2023 14:14,Təmirli deyil,Çıxarış yoxdur,İpoteka var
3,https://bina.az/items/3350837,160 000,AZN,28 May m.,2 otaqlı\n85 m²\n18/18 mərtəbə,"Bakı, bugün 14:08",17/04/2023 14:14,Təmirli,Çıxarış var,İpoteka yoxdur
4,https://bina.az/items/3410510,260 000,AZN,28 May m.,3 otaqlı\n100 m²\n19/19 mərtəbə,"Bakı, bugün 14:04",17/04/2023 14:14,Təmirli,Çıxarış var,İpoteka var
5,https://bina.az/items/3418849,144 000,AZN,Nərimanov r.,2 otaqlı\n54 m²\n2/18 mərtəbə,"Bakı, bugün 14:01",17/04/2023 14:14,Təmirli deyil,Çıxarış yoxdur,İpoteka yoxdur
6,https://bina.az/items/3418647,250 000,AZN,Nəriman Nərimanov m.,3 otaqlı\n116 m²\n4/20 mərtəbə,"Bakı, bugün 14:00",17/04/2023 14:14,Təmirli,Çıxarış var,İpoteka yoxdur
7,https://bina.az/items/3425839,420,AZN,Nəriman Nərimanov m.,4 otaqlı\n178 m²\n16/17 mərtəbə,"Bakı, bugün 14:04",17/04/2023 14:14,Təmirli deyil,Çıxarış yoxdur,İpoteka yoxdur
8,https://bina.az/items/3425843,199 000,AZN,Nəriman Nərimanov m.,3 otaqlı\n89 m²\n10/16 mərtəbə,"Bakı, bugün 13:57",17/04/2023 14:14,Təmirli deyil,Çıxarış yoxdur,İpoteka yoxdur
9,https://bina.az/items/3283753,110 000,AZN,Gənclik m.,2 otaqlı\n86 m²\n7/11 mərtəbə,"Bakı, bugün 13:50",17/04/2023 14:14,Təmirli deyil,Çıxarış yoxdur,İpoteka yoxdur


# Cleaning

In [3]:
df = pd.read_csv('bina_az_nermanov_2.0.csv').drop(columns=['Unnamed: 0']).drop_duplicates().copy()
cleaned_df=df.copy()
cleaned_df.head()

,Links,Qiymetler,Valyuta,Yer,Məlumat,Paylaşılma vaxtı,Scrape Vaxtı,Təmirli?,Çıxarış?,Ipoteka?
0,https://bina.az/items/3422474,239 000,AZN,Nəriman Nərimanov m.,2 otaqlı\n98 m²\n11/18 mərtəbə,"Bakı, bugün 23:02",16/04/2023 23:07,Təmirli,Çıxarış var,İpoteka yoxdur
1,https://bina.az/items/3353601,284 000,AZN,Nəriman Nərimanov m.,3 otaqlı\n110 m²\n3/12 mərtəbə,"Bakı, bugün 22:57",16/04/2023 23:07,Təmirli,Çıxarış var,İpoteka var
2,https://bina.az/items/3252063,389 000,AZN,Gənclik m.,3 otaqlı\n192 m²\n5/16 mərtəbə,"Bakı, bugün 22:56",16/04/2023 23:07,Təmirli,Çıxarış var,İpoteka var
3,https://bina.az/items/3391191,259 000,AZN,Nəriman Nərimanov m.,3 otaqlı\n157 m²\n4/17 mərtəbə,"Bakı, bugün 22:56",16/04/2023 23:07,Təmirli deyil,Çıxarış yoxdur,İpoteka yoxdur
4,https://bina.az/items/3402716,495 000,AZN,Nərimanov r.,4 otaqlı\n287 m²\n8/16 mərtəbə,"Bakı, bugün 22:56",16/04/2023 23:07,Təmirli,Çıxarış var,İpoteka yoxdur


In [610]:
cleaned_df['Qiymetler'] = df['Qiymetler'].apply(lambda x:x.replace(' ','')).astype(float)
cleaned_df['Elanın_Nömrəsi'] = df['Links'].apply(lambda x: 'YAŞAYIŞ KOMPLEKSLƏRİ' if 'yasayis-kompleksleri' in x else re.findall(r'/(\d+)(?:\.html)?$',x)[0])

cleaned_df['Otaq_Sayı'] = df['Məlumat'].apply(lambda x:re.findall(r'(\d{1,3})\sotaqlı\n\d{1,5}\.?\d{0,3}\sm²(?:\n\d{1,3}/\d{1,3}\smərtəbə|)?',x)[0]).astype(int)
cleaned_df['Sahə'] = df['Məlumat'].apply(lambda x:re.findall(r'\d{1,3}\sotaqlı\n(\d{1,5}\.?\d{0,3})\sm²(?:\n\d{1,3}/\d{1,3}\smərtəbə|)?',x)[0]).astype(float)
cleaned_df['Mərtəbə_raw'] = df['Məlumat'].apply(lambda x:re.findall(r'\d{1,3}\sotaqlı\n\d{1,5}\.?\d{0,3}\sm²(?:\n(\d{1,3}/\d{1,3})\smərtəbə|)?',x)[0])
cleaned_df['Mərtəbə'] = df['Məlumat'].apply(lambda x:re.findall(r'\d{1,3}\sotaqlı\n\d{1,5}\.?\d{0,3}\sm²(?:\n(\d{1,3})/\d{1,3}\smərtəbə|)?',x)[0])
cleaned_df['BinadaMaxMərtəbə'] = df['Məlumat'].apply(lambda x:re.findall(r'\d{1,3}\sotaqlı\n\d{1,5}\.?\d{0,3}\sm²(?:\n\d{1,3}/(\d{1,3})\smərtəbə|)?',x)[0])

#----------- time cleaning
def time_clean(x):
    if 'bugün' in x:
        t=re.findall(r'bugün (\d{2}:\d{2})',x)[0]
        t=datetime.date.today().strftime(f'%Y-%m-%d {t}')
    elif 'dünən' in x:
        t=re.findall(r'dünən (\d{2}:\d{2})',x)[0]
        t=(datetime.date.today()- datetime.timedelta(days=1)).strftime(f'%Y-%m-%d {t}')
    else:
        months={'yanvar':'January',
        'fevral': 'February',
        'mart':'March',
        'aprel':'April',
        'may':'May',
        'iyun':'June',
        'iyul':'July',
        'avqust':'August',
        'sentyabr':'September',
        'oktyabr':'October',
        'noyabr':'November',
        'dekabr':'December'}
        t=re.findall(r'\d{1,2} [a-z]+ \d{4}',x)[0]
        month=re.search(r'[a-z]+',t).group()
        t=t.replace(month,months[month])
        t=datetime.datetime.strptime(t,"%d %B %Y").strftime('%Y-%m-%d 00:00')
    return t
cleaned_df['Time']=df['Paylaşılma vaxtı'].apply(time_clean).apply(pd.Timestamp)


In [614]:
print(cleaned_df.shape)
cleaned_df.set_index('Time').head(10)

(4129, 17)


,Links,Qiymetler,Valyuta,Yer,Məlumat,Paylaşılma vaxtı,Scrape Vaxtı,Təmirli?,Çıxarış?,Ipoteka?,Elanın_Nömrəsi,Otaq_Sayı,Sahə,Mərtəbə_raw,Mərtəbə,BinadaMaxMərtəbə
Time,,,,,,,,,,,,,,,,
2023-04-17 23:02:00,https://bina.az/items/3422474,239000.0,AZN,Nəriman Nərimanov m.,2 otaqlı\n98 m²\n11/18 mərtəbə,"Bakı, bugün 23:02",16/04/2023 23:07,Təmirli,Çıxarış var,İpoteka yoxdur,3422474,2,98.0,11/18,11,18
2023-04-17 22:57:00,https://bina.az/items/3353601,284000.0,AZN,Nəriman Nərimanov m.,3 otaqlı\n110 m²\n3/12 mərtəbə,"Bakı, bugün 22:57",16/04/2023 23:07,Təmirli,Çıxarış var,İpoteka var,3353601,3,110.0,3/12,3,12
2023-04-17 22:56:00,https://bina.az/items/3252063,389000.0,AZN,Gənclik m.,3 otaqlı\n192 m²\n5/16 mərtəbə,"Bakı, bugün 22:56",16/04/2023 23:07,Təmirli,Çıxarış var,İpoteka var,3252063,3,192.0,5/16,5,16
2023-04-17 22:56:00,https://bina.az/items/3391191,259000.0,AZN,Nəriman Nərimanov m.,3 otaqlı\n157 m²\n4/17 mərtəbə,"Bakı, bugün 22:56",16/04/2023 23:07,Təmirli deyil,Çıxarış yoxdur,İpoteka yoxdur,3391191,3,157.0,4/17,4,17
2023-04-17 22:56:00,https://bina.az/items/3402716,495000.0,AZN,Nərimanov r.,4 otaqlı\n287 m²\n8/16 mərtəbə,"Bakı, bugün 22:56",16/04/2023 23:07,Təmirli,Çıxarış var,İpoteka yoxdur,3402716,4,287.0,8/16,8,16
2023-04-17 22:56:00,https://bina.az/items/3231011,180000.0,AZN,Nəriman Nərimanov m.,2 otaqlı\n103 m²\n6/18 mərtəbə,"Bakı, bugün 22:56",16/04/2023 23:07,Təmirli deyil,Çıxarış yoxdur,İpoteka yoxdur,3231011,2,103.0,6/18,6,18
2023-04-17 22:29:00,https://bina.az/items/3227074,299000.0,AZN,Nəriman Nərimanov m.,3 otaqlı\n129 m²\n4/17 mərtəbə,"Bakı, bugün 22:29",16/04/2023 23:07,Təmirli,Çıxarış var,İpoteka yoxdur,3227074,3,129.0,4/17,4,17
2023-04-17 22:26:00,https://bina.az/items/3411332,175000.0,AZN,Nərimanov r.,2 otaqlı\n80 m²\n14/16 mərtəbə,"Bakı, bugün 22:26",16/04/2023 23:07,Təmirli,Çıxarış var,İpoteka var,3411332,2,80.0,14/16,14,16
2023-04-17 22:23:00,https://bina.az/items/3201984,520000.0,AZN,Nəriman Nərimanov m.,3 otaqlı\n146 m²\n11/17 mərtəbə,"Bakı, bugün 22:23",16/04/2023 23:07,Təmirli,Çıxarış yoxdur,İpoteka yoxdur,3201984,3,146.0,11/17,11,17
